In [1]:
import pickle as pickle
import numpy as np
import sys
import matplotlib.pyplot as plt

print(sys.executable)

#visualize data
# for i in range(safe_samples['state'].shape[0]):
#     plt.imshow(safe_samples['state'][i])
#     plt.title(str(safe_samples['reward'][i]))
#     plt.pause(3)

/home/hyao/anaconda2/envs/tensorflow/bin/python


/home/hyao/anaconda2/envs/tensorflow/bin/python


In [ ]:
image_width, image_height = 80, 80

file = './safe_samples/safe_samples.pickle'
with open(file, 'rb') as handler:
    safe_samples = pickle.load(handler)
print(safe_samples['state'].shape)

In [2]:

x_all = safe_samples['state']
rewards = safe_samples['reward']
total_data = x_all.shape[0]
print(rewards.shape)
print(np.unique(rewards, return_counts=True))
y_vectors = np.zeros((total_data, 3), dtype = np.float32)
for i in range(total_data):
    if rewards[i] < 0:
        y_vectors[0] = 1.0
    elif rewards[i] == 0:
        y_vectors[1] = 1.0
    else:
        y_vectors[2] = 1.0

epochs_completed = 0
index_in_epoch = 0
num_examples = x_all.shape[0]
print('total number of samples is %d' % num_examples)
# serve data by batches
def next_batch(batch_size):
    global x_all
    global y_vectors
    global rewards
    global index_in_epoch
    global epochs_completed

    start = index_in_epoch
    index_in_epoch += batch_size

    # when all trainig data have been already used, it is reorder randomly
    if index_in_epoch > num_examples:
        # print('finished epoch, shuffle the data')
        epochs_completed += 1
        perm = np.arange(num_examples)
        np.random.shuffle(perm)
        x_all = x_all[perm]
        y_vectors = y_vectors[perm]
        rewards = rewards[perm]
        # start next epoch
        start = 0
        index_in_epoch = batch_size
        assert batch_size <= num_examples
    end = index_in_epoch
    # print('start=%d, %d' % (start,end))
    return x_all[start:end], y_vectors[start:end], rewards[start: end]


(10000,)
(array([-1,  0,  1]), array([3308, 3557, 3135]))
total number of samples is 10000


In [3]:

import tensorflow as tf
## multi-layer
def weight_variable(shape):
  # initial = tf.truncated_normal(shape, stddev=0.1)
  initial = tf.truncated_normal(shape, mean = 0.1, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x = tf.placeholder(tf.float32, shape=[None, image_width, image_height])
y_ = tf.placeholder(tf.float32, shape=[None, 3])

x_image = tf.reshape(x, [-1,image_width, image_height,1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
print('h_conv1: ', h_conv1.get_shape())
h_pool1 = max_pool_2x2(h_conv1)
print('h_pool1: ', h_pool1.get_shape())

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
print('h_conv2: ', h_conv2.get_shape())
h_pool2 = max_pool_2x2(h_conv2)
print('h_pool2: ', h_pool2.get_shape())

h_pool2_flat = tf.reshape(h_pool2, [-1, 20 * 20 * 64])
W_fc1 = weight_variable([20 * 20 * 64, 1024])
b_fc1 = bias_variable([1024])

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
print('h_fc1:', h_fc1.get_shape())

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
print('h_fc1_drop:', h_fc1_drop.get_shape())

W_fc2 = weight_variable([1024, 3])
b_fc2 = bias_variable([3])
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
print('y_conv:', y_conv.get_shape())


('h_conv1: ', TensorShape([Dimension(None), Dimension(80), Dimension(80), Dimension(32)]))
('h_pool1: ', TensorShape([Dimension(None), Dimension(40), Dimension(40), Dimension(32)]))
('h_conv2: ', TensorShape([Dimension(None), Dimension(40), Dimension(40), Dimension(64)]))
('h_pool2: ', TensorShape([Dimension(None), Dimension(20), Dimension(20), Dimension(64)]))
('h_fc1:', TensorShape([Dimension(None), Dimension(1024)]))
('h_fc1_drop:', TensorShape([Dimension(None), Dimension(1024)]))
('y_conv:', TensorShape([Dimension(None), Dimension(3)]))


In [5]:

sess = tf.InteractiveSession()
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())

perm = np.arange(num_examples)
np.random.shuffle(perm)
x_all = x_all[perm]
y_vectors = y_vectors[perm]
rewards = rewards[perm]
W_conv1_diff = []
losses = []
y_conv_values = []
y_true = []
for i in range(10):
    x_batch, y_batch, label_batch = next_batch(50)
    print(np.unique(label_batch, return_counts=True))
    train_accuracy = accuracy.eval(feed_dict={x: x_batch, y_: y_batch, keep_prob: 1.0})
    print('train_accuracy= ', train_accuracy)
    sess.run(train_step, feed_dict={x: x_batch, y_: y_batch, keep_prob: 1.0})
sess.close()
print(losses)

Exception AssertionError: AssertionError("Nesting violated for default stack of <type 'weakref'> objects",) in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7f10c774ce90>> ignored


(array([-1,  0,  1]), array([17, 12, 21]))
('train_accuracy= ', 1.0)
(array([-1,  0,  1]), array([13, 22, 15]))
('train_accuracy= ', 1.0)
(array([-1,  0,  1]), array([16, 19, 15]))
('train_accuracy= ', 1.0)
(array([-1,  0,  1]), array([20, 13, 17]))
('train_accuracy= ', 1.0)
(array([-1,  0,  1]), array([20, 22,  8]))
('train_accuracy= ', 1.0)
(array([-1,  0,  1]), array([22, 15, 13]))
('train_accuracy= ', 1.0)
(array([-1,  0,  1]), array([16, 20, 14]))
('train_accuracy= ', 1.0)
(array([-1,  0,  1]), array([18, 16, 16]))
('train_accuracy= ', 1.0)
(array([-1,  0,  1]), array([13, 15, 22]))
('train_accuracy= ', 1.0)
(array([-1,  0,  1]), array([15, 17, 18]))
('train_accuracy= ', 1.0)
(array([-1,  0,  1]), array([11, 22, 17]))
('train_accuracy= ', 1.0)
(array([-1,  0,  1]), array([21, 18, 11]))
('train_accuracy= ', 1.0)
(array([-1,  0,  1]), array([13, 19, 18]))
('train_accuracy= ', 1.0)
(array([-1,  0,  1]), array([17, 21, 12]))
('train_accuracy= ', 1.0)
(array([-1,  0,  1]), array([16, 1